In [2]:
from random import randint
import time
start=time.time()
class SearchEntry():
    def __init__(self, x, y, g_cost, f_cost=0, pre_entry=None):
        self.x = x
        self.y = y
        # cost move form start entry to this entry
        self.g_cost = g_cost
        self.f_cost = f_cost
        self.pre_entry = pre_entry

    def getPos(self):
        return (self.x, self.y)

class Map():
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.map = [[0 for x in range(self.width)] for y in range(self.height)]

    def createBlock(self, block_num):
        for i in range(block_num):
            x, y = (randint(0, self.width-1), randint(0, self.height-1))
            self.map[y][x] = 1

    def setBlock(self,block_list): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            self.map[y][x] = 1;
            
    def setBlock2(self,block_list): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        x_first=block_list[0][0]
        y_first=block_list[0][1]
        self.map[y_first][x_first]=1
        x_last=block_list[block_list_len-1][0]
        y_last=block_list[block_list_len-1][1]
        self.map[y_last][x_last]=1
        for i in range(1,block_list_len-1):
            x_prev=block_list[i-1][0]
            y_prev=block_list[i-1][1]
            x_next=block_list[i+1][0]
            y_next=block_list[i+1][1]
            x=block_list[i][0]
            y=block_list[i][1]
            if(abs(x_prev-x_next)==1 and abs(y_prev-y_next)==1):
                self.map[y][x]=1
            elif(self.map[y_prev][x_prev]==2):
                self.map[y][x]=1
            else:
                self.map[y][x]=2
            
    def generatePos(self, rangeX, rangeY):
        x, y = (randint(rangeX[0], rangeX[1]), randint(rangeY[0], rangeY[1]))
        while self.map[y][x] == 1:
            x, y = (randint(rangeX[0], rangeX[1]), randint(rangeY[0], rangeY[1]))
        return (x , y)

    def showMap(self):
        print("+" * (3 * self.width + 2))

        for row in self.map:
            s = '+'
            for entry in row:
                s += ' ' + str(entry) + ' '
            s += '+'
            print(s)

        print("+" * (3 * self.width + 2))
        
    def showMap2(self):
        print("+" * (3 * self.width + 2))
        len_row=len(self.map)
        #print(len_row)
        for i in range(len_row):
            s = '+'
            len_col=len(self.map[len_row-i-1])
            #print(self.map[len_row-i-1])
            for j in range(len_col):
                s +=' '+ str(self.map[len_row-i-1][j]) + ' '
            s +='+'
            print(s)

        print("+" * (3 * self.width + 2))


def AStarSearch(map, source, dest):
    def getNewPosition(map, locatioin, offset):
        x,y = (location.x + offset[0], location.y + offset[1])
        if x < 0 or x >= map.width or y < 0 or y >= map.height or map.map[y][x] == 1:
            return None
        return (x, y)

    def getPositions(map, location):#这里设置寻路类型，使用曼哈顿或折线距离
        # use four ways or eight ways to move 
        offsets = [(-1,0), (0, -1), (1, 0), (0, 1)]
        #offsets = [(-1,0), (0, -1), (1, 0), (0, 1), (-1,-1), (1, -1), (-1, 1), (1, 1)]
        poslist = []
        for offset in offsets:
            pos = getNewPosition(map, location, offset)
            if pos is not None:
                poslist.append(pos)
        return poslist

    # imporve the heuristic distance more precisely in future
    def calHeuristic(pos, dest):
        return abs(dest.x - pos[0]) + abs(dest.y - pos[1])
        
    def getMoveCost(location, pos):
        if location.x != pos[0] and location.y != pos[1]:
            return 1.4
        else:
            return 1

    # check if the position is in list
    def isInList(list, pos):
        if pos in list:
            return list[pos]
        return None
    
    # add available adjacent positions
    def addAdjacentPositions(map, location, dest, openlist, closedlist):
        poslist = getPositions(map, location)
        for pos in poslist:
            # if position is already in closedlist, do nothing
            if isInList(closedlist, pos) is None:
                findEntry = isInList(openlist, pos)
                h_cost = calHeuristic(pos, dest)
                g_cost = location.g_cost + getMoveCost(location, pos)
                if findEntry is None :
                    # if position is not in openlist, add it to openlist
                    openlist[pos] = SearchEntry(pos[0], pos[1], g_cost, g_cost+h_cost, location)
                elif findEntry.g_cost > g_cost:
                    # if position is in openlist and cost is larger than current one,
                    # then update cost and previous position
                    findEntry.g_cost = g_cost
                    findEntry.f_cost = g_cost + h_cost
                    findEntry.pre_entry = location

    # find a least cost position in openlist, return None if openlist is empty
    def getFastPosition(openlist):
        fast = None
        for entry in openlist.values():
            if fast is None:
                fast = entry
            elif fast.f_cost > entry.f_cost:
                fast = entry
        return fast

    openlist = {}
    closedlist = {}
    location = SearchEntry(source[0], source[1], 0.0)
    dest = SearchEntry(dest[0], dest[1], 0.0)
    openlist[source] = location
    while True:
        location = getFastPosition(openlist)
        if location is None:
            # not found valid path
            print("can't find valid path")
            break;
        
        if location.x == dest.x and location.y == dest.y:
            break

        closedlist[location.getPos()] = location
        openlist.pop(location.getPos())
        addAdjacentPositions(map, location, dest, openlist, closedlist)

    #mark the found path at the map
    path_temp=[]
    while location is not None:
        map.map[location.y][location.x] = 2
        #print(location.x,location.y)
        path_temp.append([location.x,location.y])
        location = location.pre_entry
    #print(path_temp)
    #print(path_temp_2)
    path=path_temp[::-1]
    #print(path_temp_3)
    return path
'''def get_path_location(map):#废弃函数
    #print(map)
    len_x=len(map)
    path=[]
    for i in range(len_x):
        len_y=len(map[i])
        for j in range(len_y):
            if(map[j][i]==2):
                path.append([i,j])
    return path'''

WIDTH = 10
HEIGHT = 10
BLOCK_NUM = 50
map = Map(WIDTH, HEIGHT)
#map.createBlock(BLOCK_NUM)
map.setBlock([[1,1],[0,0]])
#map.showMap()

source = map.generatePos((0,WIDTH//3),(0,HEIGHT//3))
dest = map.generatePos((WIDTH//2,WIDTH-1),(HEIGHT//2,HEIGHT-1))
print("source:", source)
print("dest:", dest)
path=AStarSearch(map, source, dest)
#map.showMap()
#path=get_path_location(map.map)
print(path)
map.showMap2()
end=time.time()
print(end-start)

source: (2, 1)
dest: (7, 5)
[[2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [7, 2], [7, 3], [7, 4], [7, 5]]
++++++++++++++++++++++++++++++++
+ 0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  2  0  0 +
+ 0  0  0  0  0  0  0  2  0  0 +
+ 0  0  0  0  0  0  0  2  0  0 +
+ 0  0  0  0  0  0  0  2  0  0 +
+ 0  1  2  2  2  2  2  2  0  0 +
+ 1  0  0  0  0  0  0  0  0  0 +
++++++++++++++++++++++++++++++++
0.0025110244750976562


In [6]:
#测试模块
map_width=10
map_height=10
map2=Map(map_width,map_height)
'''#map2.setBlock([[1,1],[1,2],[2,1],[2,2],[2,4],[1,4],[0,4],[2,5],[1,5],[0,5],[2,6],[1,6],[0,6],[7,3],[8,3],[7,4],[8,4]])
map2.setBlock([[1,1],[1,2],[1,3],[2,1],[2,2],[2,3],[3,1],[3,2],[3,3],[2,5],[2,6],[3,5],[3,6],[8,3],[9,3],[9,4],[8,4]])
#map2.showMap2()
source1=(4,2)
dest1=(7,3)
source2=(4,5)
dest2=(7,4)
path1=AStarSearch(map2,source1,dest1)
#map2.showMap2()
print(path1)
map2.setBlock2(path1)
#map2.showMap2()
path2=AStarSearch(map2,source2,dest2)
print(path2)
map2.setBlock2(path2)
#map2.setBlock2([[0,0]])
#map2.showMap2()
map2.showMap2()'''
source3=(0,0)
dest3=(9,7)
path3=AStarSearch(map2,source3,dest3)
map2.setBlock2(path3)
map2.showMap2()


++++++++++++++++++++++++++++++++
+ 0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  1 +
+ 0  0  0  0  0  0  0  0  0  1 +
+ 0  0  0  0  0  0  0  0  0  2 +
+ 0  0  0  0  0  0  0  0  0  1 +
+ 0  0  0  0  0  0  0  0  0  2 +
+ 0  0  0  0  0  0  0  0  0  1 +
+ 0  0  0  0  0  0  0  0  0  2 +
+ 1  2  1  2  1  2  1  2  1  1 +
++++++++++++++++++++++++++++++++
